# Load the libraries

In [15]:
import json
import os
import re
import torch
import pandas as pd
import numpy as np
import random
import time

from deep_translator import GoogleTranslator
from copy import deepcopy
from collections import defaultdict
from collections import Counter
from torch import autocast
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, \
    AutoModelForSeq2SeqLM
from tqdm import tqdm



lang_code2lang = {'es': 'Spanish'}
lang2code = {v:k for k,v in lang_code2lang.items()}

root = "/Users/pvbastidas/spanish-vqa/dataset_daquar/"


In [2]:
class CaptionDataset():
    def __init__(self, captions):
        self.captions = captions

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, index):
        return self.captions[index]

In [3]:
# Cell 2
from __main__ import CaptionDataset


In [4]:
# Define a regex pattern to normalize the question & 
# find the image ID for which the question is asked   
image_pattern = re.compile("( (in |on |of )?(the |this )?(image\d*) \?)")

with open(os.path.join(root,"all_qa_daquar.txt")) as f:
    qa_data = [x.replace("\n", "") for x in f.readlines()]

df = pd.DataFrame({"question": [], "answer": [], "image_id":[]})

raw_data = []
for i in range(0, len(qa_data), 2):
    img_id = image_pattern.findall(qa_data[i])[0][3]
    question = qa_data[i].replace(image_pattern.findall(qa_data[i])[0][0], "")
    record = {
        "question": question,
        "answer": qa_data[i+1],
        "image_id": img_id,
    }
    #df = df.append(record, ignore_index=True)
    raw_data.append(record)
print(len(raw_data))
json.dump(raw_data, open(os.path.join(root, "vqa_raw_data.json"), "w"))


12468


In [5]:
#traducir

#mc4_lang_dist = {'es': 6.593922868862283}#el 6.59 es el porcentaje vendria hacer 14133 de 214354
mc4_lang_dist = {'es': 100}#100 porciento de los datos

mc42nllb = {'es': 'spa_Latn'}
lang_code2lang = {'es': 'Spanish'}

file = "vqa_raw_data.json"

data = json.load(open(os.path.join(root, file)))
#seed = 42
#np.random.seed(seed)
#np.random.shuffle(data)

nllb_model = "facebook/nllb-200-distilled-1.3B"
model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model)
tokenizer = AutoTokenizer.from_pretrained(nllb_model)
model = model.to("cpu")

def collator(examples):
    inputs = tokenizer(examples, return_tensors="pt", padding="longest")
    return inputs
    
class CaptionDataset(Dataset):
    def __init__(self, captions):
        self.captions = captions

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, index):
        return self.captions[index]

#def translate(model, tokenizer, nllb_lang, text, batchsize = 256): original
def translate(model, tokenizer, nllb_lang, text, batchsize = 32):
    #dataloader = DataLoader(CaptionDataset(text), batch_size=batchsize, shuffle=False, num_workers=4, pin_memory=True, collate_fn=collator) original
    dataloader = DataLoader(CaptionDataset(text), batch_size=batchsize, shuffle=False, num_workers=0, pin_memory=True, collate_fn=collator)
    results = []
    for batch in tqdm(dataloader):
        batch = batch.to(model.device)
        with torch.cuda.amp.autocast():
            translated_tokens = model.generate(**batch, forced_bos_token_id=tokenizer.lang_code_to_id[nllb_lang])
        mt_prompt = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        results.extend(mt_prompt)
    return results

#funcion para reemplazar signos de pregunta pvbm
def processString(txt):
    specialChars = "¿?"
    for specialChar in specialChars:
        txt = txt.replace(specialChar, "")
    return txt


def format_answer(text):
    format = text.lower()
    format = '_'.join(format.split())
    format = format.replace(",_", ", ")
    return format


/Users/pvbastidas/miniforge3/envs/mytesisenv/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
[2024-03-16 22:12:50,242] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to mps (auto detect)


## Translate questions

In [6]:

mt_data = []
idx = 0
print(f"Total examples: {len(data)}")
for lang, part in tqdm(mc4_lang_dist.items(), total=len(mc4_lang_dist)):
    num_examples = int(len(data) * part/100)
    examples = data[idx:idx+num_examples]
    examples = deepcopy(examples)
    #time_translation(ex["answer"], "answer time traslate")
    idx = idx+num_examples
    print(f"{lang}: {num_examples} ({part}%)")
    if num_examples == 0:
        continue
    nllb_lang = mc42nllb[lang]
    question = [ex["question"] for ex in examples]
    translates_question = translate(model, tokenizer, nllb_lang, question)
    for i, ex in enumerate(examples):
        #ex["context"] = lang_code2lang[lang] + " # " + translates_question[i] original pvbm
        ex["question"] = processString(translates_question[i])
    mt_data.extend(examples)
        
print(len(mt_data))       
        
    #for ex in tqdm(examples):
        #ex["answer"] = get_translation_answer(ex["answer"])

    #mt_data.extend(examples)
with open(os.path.join(root, f"vqav2_mt_q_raw.json"), "w", encoding="utf-8") as f:
    json.dump(mt_data, f, ensure_ascii=False, indent=2)
    


Total examples: 12468


  0%|                                                     | 0/1 [00:00<?, ?it/s]

es: 12468 (100%)



  0%|                                                   | 0/390 [00:00<?, ?it/s]/Users/pvbastidas/miniforge3/envs/mytesisenv/lib/python3.8/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/pvbastidas/repoTesis/transformers/src/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|███████████████████████████████████████████| 1/1 [34:12<00:00, 2052.28s/it]

12468


In [7]:
df = pd.read_json(os.path.join(root, f"vqav2_mt_q_raw.json"))

print(df)

                                                question           answer  \
0      lo que está en el lado izquierdo del horno bla...      garbage_bin   
1      lo que está en el lado izquierdo del extintor ...            table   
2      Qué hay entre los dos contenedores de basura b...            chair   
3      Cuántos objetos hay entre el extintor de incen...                3   
4            Cuál es el objeto más grande en esta imagen  washing_machine   
...                                                  ...              ...   
12463              Qué hay debajo de la lámpara de araña            table   
12464                                Qué hay en el suelo              rug   
12465            Qué hay alrededor de la mesa de comedor            chair   
12466                    Qué hay al otro lado de la mesa  decoration_item   
12467                Qué hay detrás del divisor de pared            table   

        image_id  
0         image1  
1         image1  
2         image1  

## Translate answers

In [13]:

mt5_langs = ['es']

data = json.load(open(os.path.join(root, file)))

labels_counter = Counter(d["answer"] for d in data)
print("# answers:", len(labels_counter))
pattern = re.compile(r'\d')
labels = [l for l, num in labels_counter.items() if len(l)>1]
print("# answer:", len(labels))
#print("# labelss:", labels)

proxies = {
    "https": "13.236.6.61"
}

def translate(translator, labels):
    results = []
    bs = 130
    for i in tqdm(range(0, len(labels), bs)):
        ls = labels[i:i+bs]
        #print("lo q voy a traducir es" +str(ls))
        translations = translator.translate(text=" # ".join(ls), proxies=proxies)
        t_ls = translations.split("#")
        t_ls = [l.strip() for l in t_ls]
        
        if len(t_ls) < len(ls):
            print("Bug traducción con contexto. Se procesa el lote item por item")
            t_ls2=[];
            for l in ls:
                translations2 = translator.translate(text=(l), proxies=proxies)
                #print("esto es la cadena a traducir "+str(l)+" sto es lo q traduce" +translations2)
                t_ls2.append(translations2)
            t_ls = t_ls2;
            print(str(t_ls))
        if len(t_ls) > len(ls):
            print("Too many labels. Truncating")
            #t_ls = t_ls[:len(ls)]
        assert len(t_ls) == len(ls), f"translations {len(t_ls)}, correct {len(ls)}"
        results.extend(t_ls)
        time.sleep(1)
    return results

mt_data = dict()
print(f"Total examples: {len(data)}")

if os.path.exists("vqav2_mt_google_labels.json"):
    mt_data = json.load(open("vqav2_mt_google_labels.json"))
    done = set(mt_data.keys())
else:
    done = set()

for lang in tqdm(mt5_langs, total=len(mt5_langs)):
    target = lang
    translator = GoogleTranslator(source='en', target=target)
    #back_translator = GoogleTranslator(source=target, target="en")

    translated_labels = translate(translator, labels)
    #back_translated = translate(back_translator, translated_labels)

    keep = dict()
    for gold, mt in zip(labels, translated_labels):
        keep[gold] = mt
    print()
    print(len(keep))
    print("Total questions", sum([labels_counter[k] for k in keep.keys()]))
    print("Total questions (without yes/no)", sum([labels_counter[k] for k in keep.keys() if k not in {"yes", "no"}]))
    mt_data[lang] = keep

    with open(os.path.join(root, f"vqav2_mt_google_labels.json"), "w", encoding="utf-8") as f:
        json.dump(mt_data, f, ensure_ascii=False, indent=2)



# answers: 1443
# answer: 1434
Total examples: 12468


 25%|███████████                                 | 3/12 [00:03<00:10,  1.21s/it]

Bug traducción con contexto. Se procesa el lote item por item
['ratón', 'computadora portátil, ratón, libros, bolígrafo, papel', 'mesa, caja', 'camisas_en_percha, bolso', 'computadora portátil, teléfono, lámpara, impresora', 'marco_estante', 'silla_juguete', 'tarjetas de nota', 'taza, libros', 'taza, botella', 'album de fotos', 'pegatina', 'firmar', 'cestas_de_compras', 'monitor, papel, ratón, teclado, caja', 'chaqueta, bolso', 'perchas', '13', 'paquete_de_café, taza', 'caja, caja de pañuelos, vaso, botella, planta', 'bufanda, caja de interruptores', 'estufa_quemador', 'botella, tostadora, taza, cepillo para platos', 'lámina', 'pañuelo de papel, botella_de_liquido, cuchillo', '12', 'estante_especias, botella_especias', 'contenedor_aceite', 'grifo, fregadero', 'botella_de_liquido', 'salida de aire', 'calentador_de_agua, microondas, botella, plato', 'Reproductor de DVD', 'ropa, chaqueta', 'zapato, paraguas, bolso', 'papel, bolígrafo, reloj, caja', 'impresora, monitor, computadora portáti


 33%|██████████████▋                             | 4/12 [00:30<01:30, 11.31s/it]

Bug traducción con contexto. Se procesa el lote item por item
['papelera, libros', 'muñeca de juguete', 'muñeca', 'blanco, azul, negro, rosa, verde', 'rojo, azul, verde', 'blanco verde', 'rojo, verde, negro', 'almohada, libros', 'negro, rojo, blanco', 'teléfono inalámbrico', 'imagen, libros, papeles, elemento decorativo', 'caja de cable', 'papeles, libros', 'certificado_enmarcado, imagen', 'papel, caja de pañuelos', 'Cartas de negocios', 'paseante', 'mochila', 'hojas', 'gráfico_rollo', 'verde, azul, rojo', 'taza, bolsa', 'Teléfono móvil', 'caja_de_pañuelos, control_remoto', 'mesa_futbolín', 'dieciséis', 'morado, blanco', 'envase', 'calcetín, calcetín', 'caballo_juguete', 'equipo_de_ejercicio', 'tablero de ajedrez', 'almohada, ropa', 'xbox', 'taza, recipiente, pañuelo', 'máquina', 'rojo azul', 'casa de muñecas, monitor, impresora', 'azul, rojo, verde', 'amarillo, azul, rojo', 'bolígrafo, papel', 'tetera, plato_decorativo', 'juguete, libros', 'cesta, silla', 'crema', 'Tablón de madera', 


 58%|█████████████████████████▋                  | 7/12 [01:00<00:41,  8.38s/it]

Bug traducción con contexto. Se procesa el lote item por item
['caja, elemento_decorativo', 'taza, pote de vidrio, plato_decorativo', 'plato_decorativo, taza', 'blanca, marrón, roja, verde', 'tapete de escritorio, ropa, tejido', 'silla, lámpara', 'Lámpara de mesa', 'imagen, lámpara, florero', 'blanco, amarillo, gris', 'ropa, elemento_decorativo', 'tapete de escritorio, flor, estuche', 'amarillo, rojo, negro', 'rosa, amarillo, verde', 'imagen, imagen, imagen', 'gris blanco', 'negro, blanco, marrón, rojo', 'lámpara, mesa', 'toalla, toalla, ropa', 'imagen, lámpara', 'caja, mesa', 'cubo de basura, sofá', 'tijera, libro, papeles', 'laptop, botella, taza, papeles', 'pizarra, cartel_pizarra', 'reloj, teléfono, extintor de incendios', 'porta_archivo, portaplumas', 'tijera, bolígrafo', '22', 'papeles, reloj', 'impresora, soporte para bolígrafo, papel', 'portalápices, papeles, vidrio, botella', 'Caja electrica', 'plumero, whiteboard_marker', 'persona', 'verde, azul, amarillo, rojo', 'almohada, l


100%|█████████████████████████████████████████████| 1/1 [01:31<00:00, 91.79s/it]


1434
Total questions 10831
Total questions (without yes/no) 10831


## Generate file

In [16]:
data_q = json.load(open(os.path.join(root, "vqav2_mt_q_raw.json")))
mt_labels = json.load(open(os.path.join(root, "vqav2_mt_google_labels.json")))

vqa_data = []
mt_label_g_count = 0
for example_q in data_q:
    context = example_q["question"]
    code = "es"
    label = example_q["answer"]
    if label in mt_labels[code]:
        mt_label = mt_labels[code][label]
        label = mt_label

    vqa_data.append({
            "question": context,
            "answer": format_answer(label),
            "image_id": example_q["image_id"],
        })

with open(os.path.join(root, f"vqav2_vqa_mt_final.json"), "w", encoding="utf-8") as f:
    json.dump(vqa_data, f, ensure_ascii=False, indent=2)



## Statistics Dataset 

In [17]:
df = pd.read_json(os.path.join(root, "vqav2_vqa_mt_final.json"))
print(df) 

                                                question               answer  \
0      lo que está en el lado izquierdo del horno bla...       cubo_de_basura   
1      lo que está en el lado izquierdo del extintor ...                 mesa   
2      Qué hay entre los dos contenedores de basura b...                silla   
3      Cuántos objetos hay entre el extintor de incen...                    3   
4            Cuál es el objeto más grande en esta imagen             lavadora   
...                                                  ...                  ...   
12463              Qué hay debajo de la lámpara de araña                 mesa   
12464                                Qué hay en el suelo             alfombra   
12465            Qué hay alrededor de la mesa de comedor                silla   
12466                    Qué hay al otro lado de la mesa  elemento_decorativo   
12467                Qué hay detrás del divisor de pared                 mesa   

        image_id  
0       

In [18]:


# Create a list of all possible answers, so that the answer generation part of the VQA task 
# can be modelled as multiclass classification
answer_space = []
for ans in df.answer.to_list():
    answer_space = answer_space + [ans] if "," not in ans else answer_space + ans.replace(" ", "").split(",") 

answer_space = list(set(answer_space))
answer_space.sort()
print('num answers uniques ' +str(len(answer_space)))
with open(os.path.join(root, "answer_space.txt"), "w") as f:
    f.writelines("\n".join(answer_space))

num answers uniques 697


In [20]:
df.to_csv(os.path.join(root, "data_completo_traducido.csv"), index=None)
df2 = pd.read_csv(os.path.join(root, "data_completo_traducido.csv"))

#pd.unique(df2['question'])

#result = df2.groupby("question")["answer"].nunique().head(3)

#df2.groupby('answer', sort = False).sum() 
question_repeat=df2.groupby('question').aggregate({'answer':'count'}).sort_values(by='answer',ascending=False).head(10)
answer_repeat=df2.groupby('answer').aggregate({'question':'count'}).sort_values(by='question',ascending=False).head(10)
question_unique=df2['question'].nunique()
answer_unique=df2['answer'].nunique()
df2['question_count'] = df2['question'].str.len()


print(question_repeat)
print(answer_repeat)
print('num questions ' +str(question_unique))

print(df2.sort_values(by='question_count',ascending=True))

                                     answer
question                                   
Qué hay en la mesa                      279
Cuál es el objeto más grande            199
Cuántas sillas hay                      183
Qué hay en la pared                     162
Cuál es el objeto cerca de la pared      93
Cuántos cajones hay                      87
Qué hay en la cama                       86
Qué hay delante de la mesa               80
Qué hay en el sofá                       75
Qué hay delante del sofá                 75
          question
answer            
2              554
mesa           421
silla          367
3              327
ventana        277
blanco         259
1              252
foto           252
imagen         249
gabinete       245
num questions 5239
                                               question             answer  \
7061                                       Cuántas CPUs                  2   
8082                                      Cuántos sofás              

In [21]:
df3 = pd.read_table(os.path.join(root, "answer_space.txt"),names=["answer"])
df3['answer_count'] = df3['answer'].str.len()
print(df3.sort_values(by='answer_count',ascending=False))


                                  answer  answer_count
94   botella_de_líquido_para_lavar_manos            35
91   botella_de_liquido_para_lavar_manos            35
206   contenedor_de_juguetes_de_plástico            34
204     contenedor_de_crema_para_la_piel            32
466     paquete_de_sábanas_de_dormitorio            32
..                                   ...           ...
15                                     6             1
16                                     7             1
17                                     8             1
18                                     9             1
0                                      1             1

[697 rows x 2 columns]
